This notebook is a tutorial of the qutip-cntrl quantum optimal control library, found [here](https://github.com/qutip/qutip-qtrl). I used [this](https://nbviewer.org/github/qutip/qutip-notebooks/blob/master/examples/control-pulseoptim-CRAB-2qubitInerac.ipynb) tutorial via [QuTiP](https://qutip.readthedocs.io/en/master/index.html) to explore the package. The goal here is to calculate control fields needed to transfer a 2-Qubit system from the state 
$$
\begin{pmatrix}
1 & 0 & 0 & 0
\end{pmatrix}^\top
$$ to the state
$$
\begin{pmatrix}
0 & 0 & 0 & 1
\end{pmatrix}^\top
$$
using the Chopped Random Basis
[CRAB](https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.106.190501) algorithm.

In [58]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import shutil

In [59]:
from qutip import Qobj, identity, sigmax, sigmaz, core, tensor, identity
#QuTiP control modules
import qutip.control.pulseoptim as cpo

In [60]:
# Random seed and create variables for Pauli matrices + identity matrix for ease of use 
random.seed(20)
sx = sigmax()
sz = sigmaz()
I = identity(2)

We make use of the following combined Hamiltonian to our system:
$$H(t) = H_0 + \sum_{j=1}(u_1(t)H_{c_1} + u_2(t)H_{c_2})$$
We define our control Hamiltonian, $H_c$, as z-rotations of the 2 qubits: $$H_c = \sigma_z \otimes \sigma_z$$
We define our drift Hamiltonian, $H_d$ as $$H_d = \alpha_1(\sigma_x \otimes I) + \alpha_2(I \otimes \sigma_x) + \beta_1(\sigma_z \otimes I) + \beta_2(I \otimes \sigma_z)$$
where $\alpha_i$, $\beta_i$ are randomly initialized coefficients that are to be optimized.

In [61]:
# Define # of controls & Initialize coefficients as described above to be optimized, 
n_cntrl = 2
alpha = [random.random()] * n_cntrl
beta = [random.random()] * n_cntrl

In [62]:
# Control Hamiltonian
H_c = [tensor(sz, sz)]
# Drift Hamiltonian
H_d = (alpha[0]*tensor(sx, I)) + (alpha[1]*tensor(I, sx)) + (beta[0]*tensor(sz, I)) + (beta[1]*tensor(I,sz))

In [63]:
# Initial and target states for qubit 1 and 2 (q1, q2)
q1_0 = Qobj([[1], [0]]) # Qubit 1 initial state
q2_0 = Qobj([[1], [0]]) # Qubit 2 initial state
q1_t = Qobj([[0], [1]]) # Qubit 1 target state
q2_t = Qobj([[0], [1]]) # Qubit 2 target state
psi_0 = tensor(q1_0, q2_0) # combined initial state
psi_t = tensor(q1_t, q2_t) # combined targetr state

In [64]:
# Define evolution parameters
M = 100 # number of time slots
T = 18 # total evolution time

In [65]:
# Define thresholds for process termination
fidelity_threshold = 1e-3 # fidelity error threshold
max_iterations = 500
max_time = 120 # (seconds)

In [66]:
# Define pulse type (qutip-control options are: RND, DEF, LIN, ZERO, SINE, SQUARE, TRIANGLE, SAW)
pulse_type = 'DEF'

In [67]:
# Create an output file extension using our parameters
output_file_extension = f"CRAB_M_{M}_T_{T}_Pulse_{pulse_type}.txt"

In [68]:
# Run optimization with qutip-ctrl
result = cpo.opt_pulse_crab_unitary(H_d, H_c, psi_0, psi_t, M, T, fid_err_targ=fidelity_threshold, 
            max_iter=max_iterations, max_wall_time=max_time, init_coeff_scaling=5.0, num_coeffs=5, 
            method_params={'xtol':1e-3}, guess_pulse_type=None, guess_pulse_action='modulate',
            out_file_ext=output_file_extension, gen_stats=True)

In [69]:
# Housekeeping - Reorganize files (QuTiP-control enforces a specific filepath)
working_dir = os.getcwd() + '/' # get working directory
# Get initial amplitudes output file
initial_path_pre = 'ctrl_amps_initial_'
initial_source = working_dir + initial_path_pre + output_file_extension
# Get final amplitudes output file
final_path_pre = 'ctrl_amps_final_'
# Move both optimization output files to result directory
final_source = working_dir + final_path_pre + output_file_extension
destination = working_dir + 'optim_results/CRAB/'
shutil.move(initial_source, destination)
shutil.move(final_source, destination)

'/Users/lukeandreesen/quantum_computing/quantum_optimal_control/optim_results/CRAB/ctrl_amps_final_CRAB_M_100_T_18_Pulse_DEF.txt'